In [1]:
import pandas as pd

In [2]:
fz_seafood = pd.read_excel("/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Seafood_POS_2020.xlsx")
fz_seafood2 = pd.read_excel("/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Seafood_POS_2021.xlsx")
fz_seafood3 = pd.read_excel("/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Seafood_POS_2022.xlsx")
fz_seafood4 = pd.read_excel("/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Seafood_POS_2023.xlsx")
fz_seafood5 = pd.read_excel("/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Seafood_POS_2024.xlsx")
product_attribute = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Cleaned Data/Product Attributes.xlsx')


In [3]:
fz_seafood.head()

,Geography,Time,Product,UPC 13 digit,Unit Sales,Volume Sales,Dollar Sales,Price per Unit,Price per Volume,Base Unit Sales,Base Volume Sales,Base Dollar Sales,Incremental Units,Incremental Volume,Incremental Dollars,ACV Weighted Distribution
0,Total US - Multi Outlet + Conv,Week Ending 01-12-20,40 KNOTS SEAFOOD FROZEN SHRIMP UNCOATED SALAD ...,41172810691,368.288632,736.577265,1206.115532,3.274919,1.637460,311.969319,623.938639,1209.263738,56.319313,112.638626,-3.148206,0.196705
1,Total US - Multi Outlet + Conv,Week Ending 01-12-20,40 KNOTS SEAFOOD FROZEN SHRIMP UNCOATED WHOLE ...,41172810401,346.453916,216.533697,2931.339269,8.460979,13.537566,310.877245,194.298278,2615.065631,35.576671,22.235419,316.273638,0.202542
2,Total US - Multi Outlet + Conv,Week Ending 01-12-20,40 KNOTS SEAFOOD FROZEN SHRIMP UNCOATED WHOLE ...,41172810471,638.506363,638.506363,3824.653114,5.990000,5.990000,255.790573,255.790563,1877.194927,382.715790,382.715800,1947.458186,0.178043
3,Total US - Multi Outlet + Conv,Week Ending 01-12-20,40 KNOTS SEAFOOD FROZEN SHRIMP UNCOATED WHOLE ...,41172810481,640.083557,640.083557,5403.108594,8.441255,8.441255,371.951259,371.951258,3913.306761,268.132298,268.132299,1489.801834,0.218430
4,Total US - Multi Outlet + Conv,Week Ending 01-12-20,40 KNOTS SEAFOOD FROZEN SHRIMP UNCOATED WHOLE ...,41172810491,603.129075,603.129075,3612.743160,5.990000,5.990000,561.820393,561.820393,4952.630140,41.308682,41.308682,-1339.886981,0.224492


In [4]:
# Imputation
def imputation_process(row):
    row['Incremental Units'] = row['Unit Sales'] - row['Base Unit Sales']
    row['Incremental Volume'] = row['Volume Sales'] - row['Base Volume Sales']
    row['Incremental Dollars'] = row['Dollar Sales'] - row['Base Dollar Sales']
    return row

# Split Geography column and only save its location
def extract_location(col):
    return col.split(' - ')[0]

# convert time into year month and day
def parse_time(col):
    # Year, month, day
    sep_data = col.split('-')
    year = int('20' + sep_data[2])
    month = int(sep_data[0][-2:])
    day = int(sep_data[1])
    # Quarter of given date
    quarter = (month - 1) // 3 + 1
    return pd.Series([year, quarter, month], index = ['Year', 'Quarter', 'Month'])

# Split product column and only save company name and product name
def parse_product(col):  
    return col.split(' - ')[0]


# Based on length of company name, select product name
def extract_product_name(full_string, brand_name):
    brand_name_length = len(brand_name)
    words = full_string[brand_name_length:].strip()
    return words

# # Arrange columns order
def change_column_order(df):
    final_order = [
        'Geography', 'Year', 'Quarter', 'Month', 'Season', 'Manufacturer Name',
        'Brand Name', 'Product Name', 'Flavor / Scent', 'Meat Source',
        'Unit Sales', 'Volume Sales', 'Dollar Sales', 'Price per Unit', 
        'Price per Volume', 'Base Unit Sales', 'Base Volume Sales', 'Base Dollar Sales',
        'Incremental Units', 'Incremental Volume', 'Incremental Dollars', 
        'ACV Weighted Distribution'
    ]
    return df[final_order]

def merge_content(df1, df2):
    combined = df1.merge(df2, how = 'left', on = 'Product')
    return combined

def map_season(month):
    # Northern Hemisphere standard mapping
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'


# Full tranform process
def transform_data(df, df2):
    table1 = merge_content(df, df2)
    # Used customized function
    table1['Geography'] = table1['Geography'].map(extract_location)
    table1[['Year', 'Quarter', 'Month']] = table1['Time'].apply(parse_time)
    table1['Product'] = table1['Product'].map(parse_product)
    table1['Product Name'] = table1.apply(lambda row: extract_product_name(row['Product'], row['Brand Name']), axis=1)
    #df['Product Name'] = df.apply(extract_product_name(df['Product'], df['Brand']), axis = 1)
    
    # Mapping quarter to seasonality
    table1['Season'] = table1['Month'].apply(map_season)
    # Change column order
    transformed = change_column_order(table1).apply(imputation_process, axis = 1)
    return transformed

In [5]:
fz_seafood_2020 = transform_data(fz_seafood, product_attribute)
fz_seafood_2021 = transform_data(fz_seafood2, product_attribute)
fz_seafood_2022 = transform_data(fz_seafood3, product_attribute)
fz_seafood_2023 = transform_data(fz_seafood4, product_attribute)
fz_seafood_2024 = transform_data(fz_seafood5, product_attribute)

In [6]:
fz_seafood_2020.head()

,Geography,Year,Quarter,Brand,Product Name,Unit Sales,Volume Sales,Dollar Sales,Price per Unit,Price per Volume,Base Unit Sales,Base Volume Sales,Base Dollar Sales,Incremental Units,Incremental Volume,Incremental Dollars,ACV Weighted Distribution
0,Total US,2020,1,40 KNOTS SEAFOOD,FROZEN SHRIMP UNCOATED SALAD 32 OZ,368.288632,736.577265,1206.115532,3.274919,1.637460,311.969319,623.938639,1209.263738,56.319313,112.638626,-3.148206,0.196705
1,Total US,2020,1,40 KNOTS SEAFOOD,FROZEN SHRIMP UNCOATED WHOLE 10 OZ,346.453916,216.533697,2931.339269,8.460979,13.537566,310.877245,194.298278,2615.065631,35.576671,22.235419,316.273638,0.202542
2,Total US,2020,1,40 KNOTS SEAFOOD,FROZEN SHRIMP UNCOATED WHOLE 16 OZ,638.506363,638.506363,3824.653114,5.990000,5.990000,255.790573,255.790563,1877.194927,382.715790,382.715800,1947.458186,0.178043
3,Total US,2020,1,40 KNOTS SEAFOOD,FROZEN SHRIMP UNCOATED WHOLE 16 OZ,640.083557,640.083557,5403.108594,8.441255,8.441255,371.951259,371.951258,3913.306761,268.132298,268.132299,1489.801834,0.218430
4,Total US,2020,1,40 KNOTS SEAFOOD,FROZEN SHRIMP UNCOATED WHOLE 16 OZ,603.129075,603.129075,3612.743160,5.990000,5.990000,561.820393,561.820393,4952.630140,41.308682,41.308682,-1339.886981,0.224492


In [7]:
# Combined & export
full_fz_seafood = pd.concat([fz_seafood_2020, fz_seafood_2021, fz_seafood_2022, fz_seafood_2023, fz_seafood_2024])
output_file = '/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/full_fz_seafood.csv'
full_fz_seafood.to_csv(output_file, index=False)